# Scale Variations Investigations

## Some theory first

Let's recap: to interpolate we need only need the list of x-points $\mathbb G = \{x_j\}$ if we assume logarithmic interpolation by default, and, by default polynomial degree 4. Especially the interpolation polynomials $p_j(x)$ are uniquely defined by this. The full spell-out on the procedure is in the eko docs. Instead, let's turn our attention to the actual interpolation procedure.

First, I will introduce a more formal notation (that maybe we should inherit in the docs). The notation will consist in putting consistent indices whose order and position (up or down) matters.
- given a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$, I define $f_j = f(x_j)$, i.e. the explicit evaluation of the function $f(x)$ at an interpolation point $x_j$. As for the grid I use the notation $\{f_j\}$ to refer to all points as a set.
- given a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$, I define a new function $f^j : [0,1] \to \mathbb R : x\to f^j(x) = (f \otimes p_j)(x)$, i.e. the multiplicative convolution of $f$ with the interpolation polynomials $p_j$.
- The interpolation polynomials $p$ themselves are an exception to those definitions and I will continue to use them with a lower index, but refer to the ordinary function as defined above.
- I define the "identity function" $e : [0,1] \to \mathbb R : x\to e(x) = \delta(1-x)$, and we find $e^j(x) = p_j(x)$. Moreover, we have ${e^j}_k = (e^j)_k = \delta_{jk}$ with $\delta_{jk}$ the usual Kronecker delta due to the properties of the interpolation construction.

Now, we can write down the interpolation of a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$:
$$ f(x) \sim \bar f(x) = f_j e^j(x) $$
using Einstein summming convention as usual (Note that as in general relativity, there is an exact match between an upper and a lower index). Note that $\bar f$ can appoximate $f$ only where there points in $\mathbb G$ and since $\mathbb G$ is finite (by construction) we loose some information in the small-x regime.

Next, we can turn to yadism: the DIS cross section $\sigma(x)$ is given as the multiplicative convolution between a PDF $f(x)$ and a coefficient function $c(z)$:
$$ \sigma(x) = (f \otimes c)(x) $$
Here and in the following we neglect any additional parameters the functions may cary, such as scales.

What we actually do is a interpolated version of the master formula $\sigma(x) = f_j c^j(x)$, i.e. we compute $c^j(x)$ the convolution of the (analytic) coefficient function $c$ with the interpolation polynomial $e^j$ with respect to the (hadronic) Bjorken-x.

How about scale variations, i.e. what if $c$ itself is a convolution? No problem! Given $d = P \otimes c$ we find
$$d^k(x) = (P \otimes c \otimes e^k)(x) = (P^k \otimes c)(x) = {P^k}_j c^j(x) $$
where we approximated the well-behaved function $P^k$ with a full set of polynomials. So our strategy of computing all $c^j(x)$ first and then supply them with a convolution matrix for the splitting kernels is good.

Finally, we can turn to the Mellin transformation: given a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$, I define
$$ \tilde f(N) = \mathcal M[f(x)](N) = \int\limits_0^1 x^{N-1}f(x)\,dx $$
Note that the Mellin transformation is running over the full domain. The Mellin transformation turns multiplicative convolutions into ordinary products, so, e.g., for the DIS master formula, we find $\tilde \sigma(N) = \tilde f (N) \cdot \tilde c(N)$ (where, $\tilde\sigma$ contains information about all Bjorken-x). Specifically, using the interpolation language we find $\tilde g^j(N) = \tilde g(N) \tilde e^j(N)$.

## What happens in practice

let's prepare a yadism run - we use the simplest options:
- we compute only F2_light
- we use NLO
- we use FFNS4
- we only allow photon exchange

In [1]:
import yadism
import yaml
import numpy as np
from eko import interpolation

t = {'ID': 208,
 'PTO': 1,
 'FNS': 'FFNS',
 'DAMP': 0,
 'IC': 1,
 'IB': 0,
 'ModEv': 'TRN',
 'ModSV': 'unvaried',
 'XIR': 1.0,
 'XIF': 2.0,
 'fact_to_ren_scale_ratio': 1.0,
 'NfFF': 4,
 'MaxNfAs': 5,
 'MaxNfPdf': 5,
 'Q0': 1.65,
 'alphas': 0.118,
 'Qref': 91.2,
 'nf0': None,
 'nfref': None,
 'QED': 0,
 'alphaqed': 0.007496252,
 'Qedref': 1.777,
 'SxRes': 0,
 'SxOrd': 'LL',
 'HQ': 'POLE',
 'mc': 1.51,
 'Qmc': 1.51,
 'kcThr': 1.0,
 'mb': 4.92,
 'Qmb': 4.92,
 'kbThr': 1.0,
 'mt': 172.5,
 'Qmt': 172.5,
 'ktThr': 1.0,
 'CKM': '0.97428 0.22530 0.003470 0.22520 0.97345 0.041000 0.00862 0.04030 0.999152',
 'MZ': 91.1876,
 'MW': 80.398,
 'GF': 1.1663787e-05,
 'SIN2TW': 0.23126,
 'TMC': 0,
 'MP': 0.938,
 'Comments': 'NNPDF4.0 NLO alphas=0.118',
 'global_nx': 0,
 'EScaleVar': 1,
 'kDIScThr': 1.0,
 'kDISbThr': 1.0,
 'kDIStThr': 1.0}

o = {'PolarizationDIS': 0.0,
 'ProjectileDIS': 'electron',
 'PropagatorCorrection': 0.0,
 'TargetDIS': 'proton',
 'interpolation_is_log': True,
 'interpolation_polynomial_degree': 4,
 'interpolation_xgrid': [1.9999999999999954e-07,
  3.034304765867952e-07,
  4.6035014748963906e-07,
  6.984208530700364e-07,
  1.0596094959101024e-06,
  1.607585498470808e-06,
  2.438943292891682e-06,
  3.7002272069854957e-06,
  5.613757716930151e-06,
  8.516806677573355e-06,
  1.292101569074731e-05,
  1.9602505002391748e-05,
  2.97384953722449e-05,
  4.511438394964044e-05,
  6.843744918967896e-05,
  0.00010381172986576898,
  0.00015745605600841445,
  0.00023878782918561914,
  0.00036205449638139736,
  0.0005487795323670796,
  0.0008314068836488144,
  0.0012586797144272762,
  0.0019034634022867384,
  0.0028738675812817515,
  0.004328500638820811,
  0.006496206194633799,
  0.009699159574043398,
  0.014375068581090129,
  0.02108918668378717,
  0.030521584007828916,
  0.04341491741702269,
  0.060480028754447364,
  0.08228122126204893,
  0.10914375746330703,
  0.14112080644440345,
  0.17802566042569432,
  0.2195041265003886,
  0.2651137041582823,
  0.31438740076927585,
  0.3668753186482242,
  0.4221667753589648,
  0.4798989029610255,
  0.5397572337880445,
  0.601472197967335,
  0.6648139482473823,
  0.7295868442414312,
  0.7956242522922756,
  0.8627839323906108,
  0.9309440808717544,
  1],
 'observables': {'F2_light': [{'Q2': 300.0, 'x': 1.9999999999999954e-07},
   {'Q2': 300.0, 'x': 3.034304765867952e-07},
   {'Q2': 300.0, 'x': 4.6035014748963906e-07},
   {'Q2': 300.0, 'x': 6.984208530700364e-07},
   {'Q2': 300.0, 'x': 1.0596094959101024e-06},
   {'Q2': 300.0, 'x': 1.607585498470808e-06},
   {'Q2': 300.0, 'x': 2.438943292891682e-06},
   {'Q2': 300.0, 'x': 3.7002272069854957e-06},
   {'Q2': 300.0, 'x': 5.613757716930151e-06},
   {'Q2': 300.0, 'x': 8.516806677573355e-06},
   {'Q2': 300.0, 'x': 1.292101569074731e-05},
   {'Q2': 300.0, 'x': 1.9602505002391748e-05},
   {'Q2': 300.0, 'x': 2.97384953722449e-05},
   {'Q2': 300.0, 'x': 4.511438394964044e-05},
   {'Q2': 300.0, 'x': 6.843744918967896e-05},
   {'Q2': 300.0, 'x': 0.00010381172986576898},
   {'Q2': 300.0, 'x': 0.00015745605600841445},
   {'Q2': 300.0, 'x': 0.00023878782918561914},
   {'Q2': 300.0, 'x': 0.00036205449638139736},
   {'Q2': 300.0, 'x': 0.0005487795323670796},
   {'Q2': 300.0, 'x': 0.0008314068836488144},
   {'Q2': 300.0, 'x': 0.0012586797144272762},
   {'Q2': 300.0, 'x': 0.0019034634022867384},
   {'Q2': 300.0, 'x': 0.0028738675812817515},
   {'Q2': 300.0, 'x': 0.004328500638820811},
   {'Q2': 300.0, 'x': 0.006496206194633799},
   {'Q2': 300.0, 'x': 0.009699159574043398},
   {'Q2': 300.0, 'x': 0.014375068581090129},
   {'Q2': 300.0, 'x': 0.02108918668378717},
   {'Q2': 300.0, 'x': 0.030521584007828916},
   {'Q2': 300.0, 'x': 0.04341491741702269},
   {'Q2': 300.0, 'x': 0.060480028754447364},
   {'Q2': 300.0, 'x': 0.08228122126204893},
   {'Q2': 300.0, 'x': 0.10914375746330703},
   {'Q2': 300.0, 'x': 0.14112080644440345},
   {'Q2': 300.0, 'x': 0.17802566042569432},
   {'Q2': 300.0, 'x': 0.2195041265003886},
   {'Q2': 300.0, 'x': 0.2651137041582823},
   {'Q2': 300.0, 'x': 0.31438740076927585},
   {'Q2': 300.0, 'x': 0.3668753186482242},
   {'Q2': 300.0, 'x': 0.4221667753589648},
   {'Q2': 300.0, 'x': 0.4798989029610255},
   {'Q2': 300.0, 'x': 0.5397572337880445},
   {'Q2': 300.0, 'x': 0.601472197967335},
   {'Q2': 300.0, 'x': 0.6648139482473823},
   {'Q2': 300.0, 'x': 0.7295868442414312},
   {'Q2': 300.0, 'x': 0.7956242522922756},
   {'Q2': 300.0, 'x': 0.8627839323906108},
   {'Q2': 300.0, 'x': 0.9309440808717544},
   {'Q2': 300.0, 'x': 1.0}]},
 'prDIS': 'EM'}

new_xgrid = interpolation.make_lambert_grid(100)
#new_xgrid = np.unique(new_xgrid.tolist() + np.geomspace(1e-9,.95e-7,30).tolist())
#print(new_xgrid)
o["interpolation_xgrid"] = new_xgrid
obs = []
for x in new_xgrid:
    obs.append(dict(Q2=300,x=x))
o["observables"]["F2_light"] = obs

#with open("210.yaml","r") as f: t = yaml.safe_load(f)
#with open("test.yaml","r") as f: o = yaml.safe_load(f)
#out = yadism.output.Output.load_tar("test.tar")
out = yadism.run_yadism(t,o)
#out.dump_tar("test.tar")

┌────────────────────────────────────┐                                       
                                      │                                    │                                       
                                      │ __     __       _ _                │                                       
                                      │ \ \   / /      | (_)               │                                       
                                      │  \ \_/ /_ _  __| |_ ___ _ __ ___   │                                       
                                      │   \   / _` |/ _` | / __| '_ ` _ \  │                                       
                                      │    | | (_| | (_| | \__ \ | | | | | │                                       
                                      │    |_|\__,_|\__,_|_|___/_| |_| |_| │                                       
                                      │                                    │                                       
                                      └────────────────────────────────────┘

Plan

 • F2_light at 100 pts

Calculation

yadism took off! please stay tuned ...

...

/home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/conv.py:152: IntegrationWarning: The maximum number of 
subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  res, err = scipy.integrate.quad(

/home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/conv.py:152: IntegrationWarning: The occurrence of roundoff 
error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = scipy.integrate.quad(

[18:25:18] INFO     computing P_qq_0 - took: 1.293660 s                                      ]8;id=476814;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=558602;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py#80\80]8;;\

[18:25:19] INFO     computing P_qg_0 - took: 0.868544 s                                      ]8;id=157869;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=143744;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py#80\80]8;;\

took 13.11 s

In [2]:
new_xgrid

array([1.00000000e-07, 1.23777322e-07, 1.53208250e-07, 1.89637056e-07,
       2.34727644e-07, 2.90539545e-07, 3.59621987e-07, 4.45130326e-07,
       5.50970170e-07, 6.81975755e-07, 8.44130742e-07, 1.04484150e-06,
       1.29327537e-06, 1.60077935e-06, 1.98139827e-06, 2.45251623e-06,
       3.03565042e-06, 3.75743368e-06, 4.65083055e-06, 5.75664235e-06,
       7.12536982e-06, 8.81951827e-06, 1.09164504e-05, 1.35119161e-05,
       1.67244213e-05, 2.07006317e-05, 2.56220600e-05, 3.17133376e-05,
       3.92524449e-05, 4.85833641e-05, 6.01317219e-05, 7.44241251e-05,
       9.21120531e-05, 1.14001368e-04, 1.41088747e-04, 1.74606629e-04,
       2.16078624e-04, 2.67387760e-04, 3.30860428e-04, 4.09369497e-04,
       5.06460736e-04, 6.26507446e-04, 7.74899074e-04, 9.58270435e-04,
       1.18477905e-03, 1.46443879e-03, 1.80951850e-03, 2.23501380e-03,
       2.75919941e-03, 3.40426592e-03, 4.19704014e-03, 5.16977903e-03,
       6.36101441e-03, 7.81640704e-03, 9.58954456e-03, 1.17425884e-02,
      

In [3]:
# let's prepare the mellin transformation
from eko.interpolation import InterpolatorDispatcher
from eko.anomalous_dimensions import as1
from eko.harmonics import S1

xgrid = out["interpolation_xgrid"]

# from scipy.integrate import quad
# from scipy.interpolate import interp1d
# def mel(y, n):
#     xs = out["interpolation_xgrid"]
#     f = interp1d(xs, y)
#     return quad(lambda x: x ** (n - 1.0) * f(x), min(xs), 1.0)

interp = InterpolatorDispatcher.from_dict(out)
def mel2(ys, n):
    lnxmin = np.log(xgrid[0])
    res = 0.
    for y, bf in zip(ys, interp):
        pj = bf(n, lnxmin) * np.exp(n * lnxmin) # remember we need to counteract the x^(-N)
        res += y * pj
    return res

[18:25:30] INFO     Interpolation: number of points = 100, polynomial degree = 4, logarithmic  ]8;id=825410;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=56928;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    = True                                                                                         

In [4]:
# let's extract some elements from the operator!
# - j refers to the ESF, that is to Bjorken-x
# - the index for orders is the usual PineAPPL order 
# - the 0 thereafter refers to the operator (instead of the error)
# - next is the flavor index, where we sort antitop,...,antidown, gluon, down, ... top
#   so charm is -3 and gluon 7
# - finally I'm selecting a single basis function
locbf10 = [out["F2_light"][j].orders[(0,0,0,0)][0][-3][10] for j in range(len(xgrid))]
pqqcbf10 = [out["F2_light"][j].orders[(1,0,0,1)][0][-3][10] for j in range(len(xgrid))]
pgqgbf10 = [out["F2_light"][j].orders[(1,0,0,1)][0][7][10] for j in range(len(xgrid))]
locbf20 = [out["F2_light"][j].orders[(0,0,0,0)][0][-3][20] for j in range(len(xgrid))]
pqqcbf20 = [out["F2_light"][j].orders[(1,0,0,1)][0][-3][20] for j in range(len(xgrid))]
pgqgbf20 = [out["F2_light"][j].orders[(1,0,0,1)][0][7][20] for j in range(len(xgrid))]

In [5]:
# next, we can compute some Mellin trafos: I choosen (almost) at random the integers from N=1..10
# first the polynomials, which we can select with a sufficiently good "delta function"
mel2bf10 = np.array([mel2([0]*10 + [1] + [0]*39,n) for n in range(1,10)])
mel2bf20 = np.array([mel2([0]*20 + [1] + [0]*29,n) for n in range(1,10)])
# then all the elements
mel2lobf10 = np.array([mel2(locbf10,n) for n in range(1,10)])
mel2pqqcbf10 = np.array([mel2(pqqcbf10,n) for n in range(1,10)])
mel2pgqgbf10 = np.array([mel2(pgqgbf10,n) for n in range(1,10)])
mel2lobf20 = np.array([mel2(locbf20,n) for n in range(1,10)])
mel2pqqcbf20 = np.array([mel2(pqqcbf20,n) for n in range(1,10)])
mel2pgqgbf20 = np.array([mel2(pgqgbf20,n) for n in range(1,10)])
# finally, some anomalous dimensions
gns = np.array([as1.gamma_ns(n, S1(n)) for n in range(1, 10)])
gqg = np.array([as1.gamma_qg(n,4) for n in range(1, 10)])

### Leading order

Now, we can check leading order where we have $c(z) = e_q^2 e(z)$ and for charm (i.e. an uplike quark), we have $e_q^2 = 4/9$.

Let's recall we compute $c^j(x)$, so, e.g., we have $c^{10}(x) = {c^{10}}_j e^j(x)$ with `locbf10` = $\{{c^{10}}_j\}$, and so $\tilde c^{10}(N) = {c^{10}}_j \tilde e^j(N)$. Ultimatively we have $c^{10}(x) = 4/9{e^{10}}_j e^j(x) = 4/9 e^{10}(x)$ and so $\tilde c^{10}(N)/\tilde e^{10}(N) = 4/9$.

I'm already failing to find this! I have an additional division by $x_j$:

In [6]:
len(locbf10)

100

In [7]:
mel2lobf10/mel2bf10/xgrid[10]

array([0.44444444, 0.44444444, 0.44444444, 0.44444444, 0.44444444,
       0.44444444, 0.44444444, 0.44444444, 0.44444444])

In [8]:
mel2lobf20/mel2bf20/xgrid[20]

array([0.44444444, 0.44444444, 0.44444444, 0.44444444, 0.44444444,
       0.44444444, 0.44444444, 0.44444444, 0.44444444])

### Pqq and Pqg

Since LO is so simple we should also be able to predict the scale variation coming with $P_{qq}$ (and also with $P_{gq}$).
We have $c^j(x)=e_q^2 {P_{\text{ns}}}^j(x)$ and so $\tilde c^{10}(N) = 4/9 \tilde P_{\text{ns}}(N) \tilde e^{10}(N)$.

And here I have immediately another problem: I can not seperate the $4/9$ at $N=1$, because $P_{\text{ns}}(N=1)=0$!

In [9]:
mel2pqqcbf10/mel2bf10/xgrid[10]

array([-1.12814863, -2.47125605, -3.11549638, -3.6320239 , -4.08070734,
       -4.49707905, -4.90742212, -5.33424218, -5.79907995])

In [10]:
-4/9*gns

array([ 1.84214783e-15+0.j, -1.58024691e+00+0.j, -2.46913580e+00+0.j,
       -3.10123457e+00+0.j, -3.59506173e+00+0.j, -4.00141093e+00+0.j,
       -4.34708995e+00+0.j, -4.64808936e+00+0.j, -4.91475603e+00+0.j])

In [11]:
mel2pqqcbf10/mel2bf10/xgrid[10]/(-4/9*gns)

array([-6.12409389e+14+0.j,  1.56384172e+00-0.j,  1.26177603e+00-0.j,
        1.17115420e+00-0.j,  1.13508686e+00-0.j,  1.12387334e+00-0.j,
        1.12889822e+00-0.j,  1.14762040e+00-0.j,  1.17993242e+00-0.j])

In [12]:
mel2pgqgbf10/mel2bf10/xgrid[10]

array([0.79695075, 0.51847739, 0.40638323, 0.33616894, 0.28603671,
       0.24674108, 0.2133791 , 0.18288439, 0.15304283])

In [13]:
-4/9*gqg

array([2.37037037, 1.18518519, 0.82962963, 0.65185185, 0.54179894,
       0.46560847, 0.40917108, 0.3654321 , 0.33041526])

In [14]:
mel2pgqgbf10/mel2bf10/xgrid[10]/(-4/9*gqg/2)

array([0.67242719, 0.87493059, 0.97967386, 1.03142744, 1.05587771,
       1.05986509, 1.04298231, 1.00092135, 0.92636661])